In [21]:
pip install requests beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.


In [22]:
import requests
from bs4 import BeautifulSoup
import re

def get_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response
    else:
        print("Failed to fetch content from URL:", url)
        return None

url = 'https://www.ettoday.net/?from=logo'
response = get_content(url)
soup = BeautifulSoup(response.text, 'html.parser') #標籤樹
lates_news = soup.find_all('div', class_="piece")
#print(len(lates_news))
hrefs = []
for a in lates_news:
    try:
        href = a.find('a')['href']
        hrefs.append(href)
    except:
        continue
    #cleaned_content = re.sub(r'<.*?>', '', a.text.strip())
    #print(cleaned_content)
#print(articles[0].text)
'''
content = response.content
if content:
    print("Content fetched successfully!")
else:
    exit()
#print(content)

# 將 content 使用正則表達式清洗資料
#cleaned_content = re.sub(r'<[^>]+>', '', content.decode("utf-8"))  # 去除 HTML 標籤
cleaned_content = re.sub(r'<.*?>', '', content.decode("utf-8"))  # 去除 HTML 標籤

# 測試清洗後的內容
print("Cleaned content:")
print(cleaned_content)  # 僅列印前 1000 個字元，避免輸出過多
'''



'\ncontent = response.content\nif content:\n    print("Content fetched successfully!")\nelse:\n    exit()\n#print(content)\n\n# 將 content 使用正則表達式清洗資料\n#cleaned_content = re.sub(r\'<[^>]+>\', \'\', content.decode("utf-8"))  # 去除 HTML 標籤\ncleaned_content = re.sub(r\'<.*?>\', \'\', content.decode("utf-8"))  # 去除 HTML 標籤\n\n# 測試清洗後的內容\nprint("Cleaned content:")\nprint(cleaned_content)  # 僅列印前 1000 個字元，避免輸出過多\n'

In [ ]:
import pandas as pd


def get_article_info(url):
    response_ = get_content(url)
    soup = BeautifulSoup(response_.text, 'html.parser') #標籤樹
    title = ""
    content = ""
    author = ""
    date = ""
    try: 
        title = soup('h1', class_="title")[0].text
        story = soup('div', class_="story")[0]
        storys = story('p')
        date = soup('time', class_="date")[0].text
        for p in storys:
            if (p.text[0:2] == "記者") :
                author = p.text
            else :
                content += p.text
    except:
        i = 1
    return {'date': date, 'title': title, 'author': author, 'content': content}
data = []
for href in hrefs:
    dict = get_article_info(href)
    if (len(dict['content']) == 0):
        continue
    data.append(dict)
df = pd.DataFrame(data)
'''
def structure_content(cleaned_content):
    # 在這裡您可以根據網站內容的結構進行處理，例如提取標題、段落等
    # 這裡只是一個示例，您可以根據實際情況進行修改
    structured_data = {
        "article": [soup('a')],
        "provider": [soup('div', class_="vr1PYe")],
        "time": [soup('time', class_="hvbAAd")]
    }
    dataframe = pd.DataFrame(structured_data)
    return dataframe, structured_data

dataframe, structured_data = structure_content(cleaned_content)
print("DataFrame:")
print(dataframe.head())

print("\nDictionary:")
print(structured_data)
'''


In [ ]:
df.head()

In [ ]:
# 輸出 CSV 檔案
df.to_csv("et_today.csv", index=False)

# 輸出 JSON 檔案
import json
with open("et_today.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False)

print("CSV and JSON files saved successfully!")


In [ ]:
df.head()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from community import community_louvain

import jieba
import jieba.analyse
import re

In [ ]:
def keep_chinese_chars(text):
    pattern = re.compile(r'[^\u4e00-\u9fff]')
    chinese_text = re.sub(pattern, '', text)
    return chinese_text

def extract_keywords(text):
    text = keep_chinese_chars(text)
    return jieba.analyse.extract_tags(text, topK=10)

df['keywords'] = df['content'].apply(extract_keywords)

In [ ]:
keywords

In [ ]:
#!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download
import matplotlib as mpl
from matplotlib.font_manager import fontManager
# 改style要在改font之前
# plt.style.use('seaborn')
fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

In [ ]:
def draw(df):
    G = nx.Graph()

    for i, row in df.iterrows():
        #content = row['content']
        author = row['author']
        keywords = row['keywords']

        G.add_node(author, type='author')

        for keyword in keywords:
            G.add_node(keyword, type='keyword')
            G.add_edge(author, keyword)
    partition = community_louvain.best_partition(G)

    community_colors = [partition[node] for node in G.nodes()]

    cmap = plt.cm.jet
    colors_with_alpha = [cmap(community_color / max(community_colors)) for community_color in community_colors]

    colors_with_alpha = [(r, g, b, 0.3) for r, g, b, _ in colors_with_alpha]

    node_sizes = [200 * G.degree(node) for node in G.nodes()]

    pos = nx.spring_layout(G, k=0.2, iterations=80)

    plt.figure(figsize=(30, 30))
    nx.draw_networkx_edges(G, pos, alpha=0.5)
    nx.draw_networkx_nodes(G, pos, node_color=colors_with_alpha, node_size=node_sizes, cmap=plt.cm.jet)
    nx.draw_networkx_labels(G, pos, font_size=20, font_family='Taipei Sans TC Beta')

    plt.axis('off')
    plt.show()

In [ ]:
draw(df)

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer


In [ ]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import torch

def bart_summarize(text):

    keep_chinese_chars(text)

    tokenizer = BertTokenizer.from_pretrained('hfl/chinese-bert-wwm')
    model = BertForMaskedLM.from_pretrained('hfl/chinese-bert-wwm')

    inputs = tokenizer(text, return_tensors="pt", max_length=100, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=128)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
df['content'] = df['content'].apply(bart_summarize)
df['keywords'] = df['content'].apply(extract_keywords)
draw(df)